# Reproducing `simulator` package Quick Start

This tutorial reimplements the execution part of [Getting Started with Simulator](http://faculty.bscb.cornell.edu/~bien/simulator_vignettes/getting-started.html) example. We demonstrate in this tutorial how compound module executables are specified with scripts and inline executables.

Material to run this tutorial can be found in [DSC2 vignettes repo](https://github.com/stephenslab/dsc2/tree/master/vignettes/simulator_example). It also contains the code to reproduce the original `simulator` simulation.

## DSC Specification
The problem is fully specified in DSC2 syntax below:

```
simulate: model.R + R(m = simulate(n, prob))
  seed: R(1:10)
  n: 50
  prob: R(seq(0,1,length=6))
  $model: m

my_method, their_method: (my.R, their.R) + R(fit = method($(model)$x)$fit)
  $fit: fit

abs, mse: (herloss.R, hisloss.R) + R(score = metric($(model)$mu, $(fit)))
  $score: score

DSC:
  define:
    method: my_method, their_method
    score: abs, mse
  run: simulate * method * score
  output: simulator_results
  exec_path: R
```

## Run DSC

In [1]:
%cd ~/GIT/dsc2/vignettes/simulator_example

/home/gaow/GIT/dsc2/vignettes/simulator_example

In [2]:
! dsc main.dsc

INFO: Checking R library dscrutils@stephenslab/dsc2/dscrutils ...
INFO: DSC script exported to simulator_results.html
INFO: Constructing DSC from main.dsc ...
INFO: Building execution graph & running DSC ...
DSC: 100%|████████████████████████████████████████| 8/8 [01:02<00:00,  8.24s/it]
INFO: Building DSC database ...
INFO: DSC complete!
INFO: Elapsed time 67.413 seconds.


You may notice that for this trivial benchmark, the execution is a lot slower compared to the original implementation. This is a limitation of DSC2 when running lite jobs -- having to build and execute DAG, checking file signatures and monitoring runtime environment results in substential overhead. For real-world benchmarking when execution time is much longer, this overhead is perhaps tolerable because it is relatively insignificant under such scenario. Also notice that under `simulator_results` there are 420 intermediate output `*.rds` files generated, compared to only 31 `*.Rdata` itermediate files generated when you run the `simulator` version of the example. This is because the demonstrated DSC2 implementation is a lot more modular. You can compare the data generating code in [DSC2](https://github.com/stephenslab/dsc2/blob/master/vignettes/simulator_example/R/model.R) vs in [`simulator`](https://github.com/stephenslab/dsc2/blob/master/vignettes/simulator_example/sims/model_functions.R) to tell the more modular setup we have adopted here -- although it is entirely possible to develop a different style of DSC that reproduces exactly the `simulator` example's output (yet stored in `rds` rather than `Rdata` format). 